Advanced altimetric computations in SMRT
==========================================

Here we show how to run the altimetric computation with a multi-layered snowpack having a rough surface

In [ ]:
import numpy as np
from smrt import make_snowpack,  make_interface, make_model, make_soil, GHz

from smrt.inputs.altimeter_list import saral_altika, envisat_ra2

%matplotlib widget
import matplotlib.pyplot as plt

Let define the snowpack and add a rough surface using the IEM model

In [ ]:
thickness = [0.5, 0.5, 2, 10]
radius = [300e-6, 400e-6, 500e-6, 700e-6]
density = [250, 300, 350, 400]

iem = make_interface("iem_fung92_brogioni10", roughness_rms=0.6e-2, corr_length=5e-2)

snowpack = make_snowpack(thickness, "sticky_hard_spheres", radius=radius, density=density, temperature=270, interface=iem)



In [ ]:
sensor = envisat_ra2('Ku')

m = make_model("iba", "nadir_lrm_altimetry")

res = m.run(sensor, snowpack)


In [ ]:
plt.figure()
plt.plot(res.t_gate * 1e9, res.sigma())
plt.xlabel('Time (ns)')
plt.ylabel('Backscatter')


We often want to split the contributions from the surface, volume and internal interfaces. This is done with an option to the RT solver.  See nadir_lrm_altimetry for other (less usefull) options.

In [ ]:
m = make_model("iba", "nadir_lrm_altimetry",
               rtsolver_options=dict(return_contributions=True))

res = m.run(sensor, snowpack)

In [ ]:
plt.figure()
plt.plot(res.t_gate * 1e9, res.sigma(contribution='total'), 'k', label='total')
plt.plot(res.t_gate * 1e9, res.sigma(contribution='volume'), label='volume')
plt.plot(res.t_gate * 1e9, res.sigma(contribution='surface'), label='surface')
plt.plot(res.t_gate * 1e9, res.sigma(contribution='interfaces'), label='interface')
plt.xlabel('Time (ns)')
plt.ylabel('Backscatter')
plt.legend()